# Task 6 : 从数据集中提取样本

In [1]:
import os,json,sys,logging
sys.path.append("./share")
sys.path.append("./common")
import pandas as pd
import json
from tqdm import tqdm
from IoTCommon import CIoTCommon
from IoTTotalFeature import CIoTTotalFeature
from IoTSample import CIoTSample
from Config import g_data_root
g_sample_root = "%ssample"%g_data_root

### 检测数据集类型

In [2]:
df_features = pd.read_excel("%s/features/features.xlsx"%g_data_root,index_col=0).reset_index()
df_features = df_features[df_features['effective'] !="N"]
df_features = df_features[df_features['count'] > 1].reset_index(drop=True)
totalFeature = CIoTTotalFeature()
df_features = CIoTTotalFeature.get_expand_features(df_features,totalFeature)

In [ ]:
def check_columns(trafficType,protocol,feature,type):
    mask=df_features['feature']==feature
    df_tmp = df_features[mask]
    tmp = {}
    tmp['trafficType'] = trafficType
    tmp['protocol'] = protocol
    tmp['feature'] = feature
    if df_tmp.shape[0] <= 0:
        tmp['result'] = "Not Found"
    else:
        tmp['configure_type']=df_tmp.iloc[0]['type']
        if tmp['configure_type'] in ['int']:
            tmp['configure_type'] = "int64"
            
        if tmp['configure_type'] in ['binary','string']:
            tmp['configure_type'] = "object"
            
        tmp['current_type'] = type
        tmp['values']=df_tmp.iloc[0]['values']
        tmp['result'] = "succeed" if tmp['current_type'] == tmp['configure_type'] else "failed" 
    
    return tmp
    
all_data = []
test = CIoTSample()
for trafficType in test.get_attack_type():
    for protocol in tqdm(test.get_attack_protocol(trafficType),desc="loading attack samples"):
        df_test = test.get_attack_sample(trafficType,protocol)
        for col in df_test.keys():
            type = df_test.dtypes[col]
            tmp = check_columns(trafficType,protocol,col,type)
            all_data.append(tmp)
    
for trafficType in test.get_sensor_type():
    for protocol in tqdm(test.get_sensor_protocol(trafficType),desc="loading normal samples"):
        df_test = test.get_sensor_sample(trafficType,protocol)
        for col in df_test.keys():
            type = df_test.dtypes[col]
            tmp = check_columns(trafficType,protocol,col,type)
            all_data.append(tmp)
            
df_result = pd.DataFrame(all_data)

loading attack samples:  43%|█████████████████████████████████████████▌                                                       | 3/7 [03:44<05:34, 83.50s/it]

In [ ]:
df_tmp =df_result[df_result['configure_type'] !=df_result['current_type']].dropna().groupby(['result','trafficType','protocol','feature','configure_type','current_type']).count().reset_index()

In [ ]:
df_tmp[~df_tmp['configure_type'].isin(['datetime','category'])]

In [ ]:
df_tmp.to_csv("test.csv")

In [ ]:
trafficType="phValue"
protocol="eth:ethertype:ip:tcp:mqtt"
feature = 'mqtt.connack.reason_code'
df_test = test.get_sensor_sample(trafficType,protocol)
df_test[feature]
